<a href="https://colab.research.google.com/github/TiagoIesbick/dashboard-etl/blob/main/dashboard_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import pathlib

In [13]:
file_path = r'/content/drive/MyDrive/Dashboard_data/utils/auxiliary_tables.xlsx'

sheets_info = {
    'n_uni': {'skiprows': 2, 'cols': ['uni']},
    'n_proj': {'skiprows': 3, 'cols': ['proj', 'nome_proj']},
    'n_elem': {'skiprows': 3, 'cols': ['elem', 'nome_elem']},
    'n_vinc': {'skiprows': 3, 'cols': ['vinc', 'nome_vinc']},
    'n_rub': {'skiprows': 3, 'cols': ['rub', 'nome_rub']}
}

dfs = {
    sheet: pd.read_excel(file_path, sheet_name=sheet, skiprows=info['skiprows'], header=None, names=info['cols'])
    for sheet, info in sheets_info.items()
}

for sheet, df in dfs.items():
  if 'uni' in df.columns:
      df['nome_uni'] = df['uni'].str[5:]
      df['uni_int'] = df['uni'].str[:4].astype(int)
  if 'proj' in df.columns:
      df['proj_int'] = df['proj'].str[:4].astype(int)
  if 'elem' in df.columns:
      df['elem_int'] = df['elem'].str[:6].astype(int)
  if 'rub' in df.columns:
      df['rub_int'] = df['rub'].str[:12].astype('int64')
  if 'vinc' in df.columns:
      df['vinc_int'] = df['vinc'].str[:4].astype(int)

df_uni, df_proj, df_elem, df_vinc, df_rub = dfs.values()

In [36]:
# Generating a dataframe with paid expenses data from 2002 to 2023

exp_folder_2002_2023 = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/Despesas/2002_2023')

df_list = [
    pd.read_excel(expense_file).dropna(axis=1, how='all')
    for expense_file in exp_folder_2002_2023.iterdir()
]

df_exp_2002_2023 = pd.concat(df_list, ignore_index=True)

df_exp_2002_2023 = df_exp_2002_2023[['CPTPAG', 'RESUL_PAGO', 'UNIDORC', 'PROJATIV', 'RUBRICA', 'VINCORC']].rename(columns={
    'CPTPAG':'Comp.pagto.', 'RESUL_PAGO':'Result. pago', 'UNIDORC':'Unid. Orçam.', 'PROJATIV':'Proj/Ativ',
    'RUBRICA':'Rubrica', 'VINCORC':'Vinc. Orçam.'
})

df_exp_2002_2023 = df_exp_2002_2023.dropna(subset=['Comp.pagto.', 'Result. pago'], ignore_index=True)
df_exp_2002_2023['Elemento'] = df_exp_2002_2023['Rubrica'].astype(str).str[:6].astype(int)

In [29]:
# # Define the folder path
# exp_folder_2024_onwards = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/Despesas/2024+')

# # Read all CSV files efficiently
# df_list = [
#     pd.read_csv(file, sep=',', dtype=str).dropna(axis=1, how='all')
#     for file in exp_folder_2024_onwards.glob("*.csv")
# ]

# # Concatenate all DataFrames
# df_exp_2024_onwards = pd.concat(df_list, ignore_index=True)

# # Convert financial columns efficiently
# cols_to_convert = ['valorpago', 'restospagarnaoprocessadospagos', 'restospagarprocessadospagos']
# df_exp_2024_onwards[cols_to_convert] = df_exp_2024_onwards[cols_to_convert].apply(
#     lambda col: pd.to_numeric(col.str.replace('R\$ ', '', regex=True).str.replace(',', '.'), errors='coerce')
# )

# # Filter out rows with zero values in all target columns
# df_exp_2024_onwards = df_exp_2024_onwards[
#     df_exp_2024_onwards[cols_to_convert].ne(0).any(axis=1)
# ]

# # Rename and clean up columns
# df_exp_2024_onwards.rename(columns={
#     'exercicio': 'YEAR', 'mes': 'MONTH', 'subacao': 'Proj/Ativ', 'unidadeorcamentaria': 'Unid. Orçam.'
# }, inplace=True)

# # Fix month values
# df_exp_2024_onwards['MONTH'] = df_exp_2024_onwards['MONTH'].replace({0: 1, 13: 12}).astype(int)

# # Convert date
# df_exp_2024_onwards['Comp.pagto.'] = pd.to_datetime(df_exp_2024_onwards[['YEAR', 'MONTH']].assign(DAY=1))

# # # Process categorical columns efficiently
# df_exp_2024_onwards['Elemento'] = df_exp_2024_onwards['elementocompleto'].str[:8].str.replace('.', '', regex=False).astype(int)
# df_exp_2024_onwards['Rubrica'] = df_exp_2024_onwards['desdobramentocompleto'].str[:12].str.replace('.', '', regex=False).astype(int)

# # Sorting once before processing
# df_exp_2024_onwards.sort_values('Comp.pagto.', inplace=True, ignore_index=True)

# # Subtract the last nonzero appearance for both columns
# cols = ['Comp.pagto.', 'Proj/Ativ', 'Rubrica', 'fonterecursos']
# for target_col in ['restospagarnaoprocessadospagos', 'restospagarprocessadospagos']:
#     df_exp_2024_onwards[target_col + '_prev'] = (
#         df_exp_2024_onwards.groupby(cols)[target_col].shift(1).fillna(0)
#     )
#     df_exp_2024_onwards[target_col] -= df_exp_2024_onwards[target_col + '_prev']
#     df_exp_2024_onwards[target_col] = df_exp_2024_onwards[target_col].clip(lower=0)  # Avoid negatives

# # Compute final results
# df_exp_2024_onwards['Result. pago'] = df_exp_2024_onwards[cols_to_convert].sum(axis=1)

# # Final filter and reordering columns
# df_exp_2024_onwards = df_exp_2024_onwards[df_exp_2024_onwards['Result. pago'] != 0].reset_index(drop=True)
# df_exp_2024_onwards['Vinc. Orçam.'] = df_exp_2024_onwards['defonterecursos'].str[:4].astype(int)

# df_exp_2024_onwards = df_exp_2024_onwards[
#     ['Comp.pagto.', 'Result. pago', 'Unid. Orçam.', 'Proj/Ativ', 'Elemento', 'Rubrica', 'Vinc. Orçam.',
#      'deelemento', 'dedesdobramento', 'desubacao', 'deunidadeorcamentária', 'defonterecursos']
# ]

In [33]:
# Generating a dataframe with paid expenses data from 2024 onwards

exp_folder_2024_onwards = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/Despesas/2024+')

df_list = [
    pd.read_csv(expense_file).dropna(axis=1, how='all')
    for expense_file in exp_folder_2024_onwards.iterdir()
]

df_exp_2024_onwards = pd.concat(df_list, ignore_index=True)

cols_to_convert = ['valorpago', 'restospagarnaoprocessadospagos', 'restospagarprocessadospagos']

df_exp_2024_onwards[cols_to_convert] = (
    df_exp_2024_onwards[cols_to_convert]
    .apply(lambda col: col.str.removeprefix('R$ ').str.replace(',', '.', regex=False).astype(float))
)

df_exp_2024_onwards = df_exp_2024_onwards.loc[
    (df_exp_2024_onwards['valorpago'] != 0) |
    (df_exp_2024_onwards['restospagarnaoprocessadospagos'] != 0) |
    (df_exp_2024_onwards['restospagarprocessadospagos'] != 0)
]

df_exp_2024_onwards['mes'] = df_exp_2024_onwards['mes'].replace({0: 1, 13: 12})
df_exp_2024_onwards.rename(columns={
    'exercicio': 'YEAR', 'mes': 'MONTH', 'subacao': 'Proj/Ativ', 'unidadeorcamentaria': 'Unid. Orçam.'
    }, inplace=True)
df_exp_2024_onwards['Comp.pagto.'] = pd.to_datetime(df_exp_2024_onwards[['YEAR', 'MONTH']].assign(DAY=1))
df_exp_2024_onwards['Elemento'] = df_exp_2024_onwards['elementocompleto'].str[:8].str.replace('.', '', regex=False).astype(int)
df_exp_2024_onwards['Rubrica'] = df_exp_2024_onwards['desdobramentocompleto'].str[:12].str.replace('.', '', regex=False).astype(int)
df_exp_2024_onwards = df_exp_2024_onwards.sort_values('Comp.pagto.').reset_index(drop=True)

cols = ['Comp.pagto.', 'Proj/Ativ', 'Rubrica', 'fonterecursos']
target_cols = ['restospagarnaoprocessadospagos', 'restospagarprocessadospagos']

for target_col in target_cols:
    df_aux = df_exp_2024_onwards.loc[df_exp_2024_onwards[target_col] != 0, cols + [target_col]]
    df_exp_2024_onwards[target_col + '_aux'] = df_exp_2024_onwards[target_col]

    for i in df_aux.index:
        df = df_exp_2024_onwards.loc[
            (df_exp_2024_onwards['Comp.pagto.'] < df_aux.loc[i, 'Comp.pagto.']) &
            (df_exp_2024_onwards['Comp.pagto.'].dt.year == df_aux.loc[i, 'Comp.pagto.'].year) &
            (df_exp_2024_onwards['Proj/Ativ'] == df_aux.loc[i, 'Proj/Ativ']) &
            (df_exp_2024_onwards['Rubrica'] == df_aux.loc[i, 'Rubrica']) &
            (df_exp_2024_onwards['fonterecursos'] == df_aux.loc[i, 'fonterecursos']) &
            (df_exp_2024_onwards[target_col] != 0), target_col + '_aux']

        if not df.empty:
            df_exp_2024_onwards.loc[i, target_col] -= df.iloc[-1]

df_exp_2024_onwards['Result. pago'] = df_exp_2024_onwards[cols_to_convert].sum(axis=1, numeric_only=True)

df_exp_2024_onwards = df_exp_2024_onwards.loc[df_exp_2024_onwards['Result. pago'] != 0].reset_index(drop=True)
df_exp_2024_onwards['Vinc. Orçam.'] = df_exp_2024_onwards['defonterecursos'].str[:4].astype(int)
df_exp_2024_onwards = df_exp_2024_onwards[[
    'Comp.pagto.', 'Result. pago', 'Unid. Orçam.', 'Proj/Ativ', 'Elemento', 'Rubrica', 'Vinc. Orçam.',
    'deelemento', 'dedesdobramento', 'desubacao', 'deunidadeorcamentária', 'defonterecursos'
    ]]

In [16]:
# concatenating the expense dataframes
relevant_columns = ['Comp.pagto.', 'Result. pago', 'Unid. Orçam.', 'Proj/Ativ', 'Elemento', 'Rubrica', 'Vinc. Orçam.']
df_exp = pd.concat([df_exp_2002_2023, df_exp_2024_onwards[relevant_columns]], ignore_index=True)

In [17]:
# Updating the elements dataframe

df_new_elem = df_exp_2024_onwards[['Elemento', 'deelemento']].drop_duplicates(keep='last').rename(columns={
    'Elemento': 'elem_int',
    'deelemento': 'nome_elem'
})
df_elem = pd.concat([df_elem, df_new_elem], ignore_index=True).drop_duplicates(subset='elem_int', keep='last').reset_index(drop=True)
df_elem['elem'] = df_elem['elem_int'].astype(str) + ' - ' + df_elem['nome_elem'].str.title()

In [18]:
# Updating the rubric dataframe

df_new_rub = df_exp_2024_onwards[['Rubrica', 'dedesdobramento']].drop_duplicates(keep='last').rename(columns={
    'Rubrica': 'rub_int',
    'dedesdobramento': 'nome_rub'
})
df_rub = pd.concat([df_rub, df_new_rub], ignore_index=True).drop_duplicates(subset='rub_int', keep='last').reset_index(drop=True)
df_rub['rub'] = df_rub['rub_int'].astype(str) + ' - ' + df_rub['nome_rub'].str.title()

In [19]:
# Updating the project dataframe

df_new_proj = df_exp_2024_onwards[['Proj/Ativ', 'desubacao']].drop_duplicates(keep='last').rename(columns={
    'Proj/Ativ': 'proj_int',
    'desubacao': 'nome_proj'
})
df_proj = pd.concat([df_proj, df_new_proj], ignore_index=True).drop_duplicates(subset='proj_int', keep='last').reset_index(drop=True)
df_proj['proj'] = df_proj['proj_int'].astype(str) + ' - ' + df_proj['nome_proj'].str.title()

In [20]:
# Updating the budget units dataframe

df_new_uni = df_exp_2024_onwards[['Unid. Orçam.', 'deunidadeorcamentária']].drop_duplicates(keep='last').rename(columns={
    'Unid. Orçam.': 'uni_int',
    'deunidadeorcamentária': 'nome_uni'
})
df_uni = pd.concat([df_uni, df_new_uni], ignore_index=True).drop_duplicates(subset='uni_int', keep='last').reset_index(drop=True)
df_uni['uni'] = df_uni['uni_int'].astype(str) + ' - ' + df_uni['nome_uni'].str.title()

In [21]:
# Updating the entail dataframe

df_new_vinc = df_exp_2024_onwards[['Vinc. Orçam.', 'defonterecursos']].drop_duplicates(keep='last').rename(columns={
    'Vinc. Orçam.': 'vinc_int', 'defonterecursos': 'nome_vinc'
})
df_new_vinc['nome_vinc'] = df_new_vinc['nome_vinc'].str.split('-').str.get(1).str.strip()
df_vinc = pd.concat([df_vinc, df_new_vinc], ignore_index=True).drop_duplicates(subset='vinc_int', keep='last').reset_index(drop=True)
df_vinc['vinc'] = df_vinc['vinc_int'].astype(str) + ' - ' + df_vinc['nome_vinc'].str.title()

In [22]:
final_data_folder = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/final_data')

df_uni.sort_values('uni_int').to_csv(final_data_folder / 'df_uni.csv', encoding='utf-8-sig', sep=';', index=False)
df_proj.sort_values('proj_int').to_csv(final_data_folder / 'df_proj.csv', encoding='utf-8-sig', sep=';', index=False)
df_elem.sort_values('elem_int').to_csv(final_data_folder / 'df_elem.csv', encoding='utf-8-sig', sep=';', index=False)
df_rub.sort_values('rub_int').to_csv(final_data_folder / 'df_rub.csv', encoding='utf-8-sig', sep=';', index=False)
df_vinc.sort_values('vinc_int').to_csv(final_data_folder / 'df_vinc.csv', encoding='utf-8-sig', sep=';', index=False)
df_exp.sort_values('Comp.pagto.').to_csv(final_data_folder / 'df_exp.csv', encoding='utf-8-sig', sep=';', index=False)

**Revenue**

In [ ]:
# Generating a dataframe with revenue collected from 2004 to 2017

revenue_file_2004_2017 = r'/content/drive/MyDrive/Dashboard_data/Receitas/2004_2017/2004_2017_Cubo_Antigo.xlsx'

df_rev_2004_2017 = pd.read_excel(revenue_file_2004_2017)
df_rev_2004_2017.drop(0, inplace=True)
df_rev_2004_2017.rename(columns={'Exercício': 'YEAR',
                                 'Mês': 'MONTH',
                                 'Rótulos de Linha': 'orgao',
                                 'Vínculo Cod': 'vinculo',
                                 'N6 Subalinea': 'desdobramento6',
                                 'Valor Arrecadado': 'valor_arrecadado',
                                 'Valor Orçado': 'valor_orcado'}, inplace=True)
df_rev_2004_2017['Data'] = pd.to_datetime(df_rev_2004_2017[['YEAR', 'MONTH']].assign(DAY=1))
df_rev_2004_2017[['YEAR', 'orgao', 'MONTH', 'vinculo']] = df_rev_2004_2017[['YEAR', 'orgao', 'MONTH', 'vinculo']].astype('int64')
df_rev_2004_2017['vinculo'] = df_rev_2004_2017['vinculo'].astype('str').replace('400', '0400').replace('6051', '605X')\
  .replace('1', '0001').replace('605X', '6051')

In [ ]:
# Generating a dataframe with revenue collected from 2018 to 2023

rev_folder_2018_2023 = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/Receitas/2018_2023')

df_list = [
    pd.read_excel(revenue_file).dropna(axis=1, how='all')
    for revenue_file in rev_folder_2018_2023.iterdir()
]

df_rev_2018_2023 = pd.concat(df_list, ignore_index=True)

df_rev_2018_2023.drop(columns=['valor_cancelado', 'valor_lancado', 'valor_meta', 'informacao_complementar'], inplace=True)
df_rev_2018_2023 = df_rev_2018_2023.loc[df_rev_2018_2023['orgao_raiz'] == 7000].reset_index(drop=True)
df_rev_2018_2023.rename(columns={'ano': 'YEAR', 'mes': 'MONTH'}, inplace=True)
df_rev_2018_2023['Data'] = pd.to_datetime(df_rev_2018_2023[['YEAR', 'MONTH']].assign(DAY=1))
df_rev_2018_2023['vinculo'] = df_rev_2018_2023['vinculo'].astype(str).replace({'400': '0400'})
df_rev_2018_2023['rubrica'] = df_rev_2018_2023['rubrica'].astype(str)

df_rev_2018_2023['Natureza da Receita Antiga'] = df_rev_2018_2023['rubrica'].str.pad(14, fillchar='0').str.replace(
    r'(\d{2})(\d)(\d)(\d)(\d{2})(\d)(\d)(\d{2})(\d{2})(\d+)', r'\1.\2.\3.\4.\5.\6.\7.\8.\9.\10', regex=True
) + '.' + df_rev_2018_2023['vinculo'] + '.0'

In [ ]:
# Generating a dataframe with revenue collected from 2024 onwards

rev_folder_2024_onwards = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/Receitas/2024+')

df_list = [
    pd.read_csv(revenue_file).dropna(axis=1, how='all')
    for revenue_file in rev_folder_2024_onwards.iterdir()
]

df_rev_2024_onwards = pd.concat(df_list, ignore_index=True)

df_rev_2024_onwards.rename(columns={'exercicio': 'YEAR', 'mes': 'MONTH'}, inplace=True)
df_rev_2024_onwards['Data'] = pd.to_datetime(df_rev_2024_onwards[['YEAR', 'MONTH']].assign(DAY=1))

In [ ]:
df_exp_2002_2023.loc[df_exp_2002_2023['Comp.pagto.'].dt.year == 2023, 'Result. pago'].sum()

1850336804.21

In [33]:
df_exp.loc[df_exp['Comp.pagto.'].dt.year == 2025, 'Result. pago'].sum()

148988461.28000003

In [24]:
df_exp

,Comp.pagto.,Result. pago,Unid. Orçam.,Proj/Ativ,Rubrica,Vinc. Orçam.,Elemento
0,2019-01-15,430.00,7001,2870,339039720100,6069,339039
1,2019-01-21,430.00,7001,2870,339039720100,6069,339039
2,2019-01-29,11666.33,7001,2870,339039720100,6069,339039
3,2019-02-04,215.00,7001,2870,339039720100,6069,339039
4,2019-02-11,215.00,7001,2870,339039720100,6069,339039
...,...,...,...,...,...,...,...
51601,2025-02-01,1637.04,7002,9074,31909130,6049,319091
51602,2025-02-01,1928.11,7002,9074,31909128,6049,319091
51603,2025-02-01,74.72,7002,2747,31909201,6049,319092
51604,2025-02-01,6310.47,7002,2740,31909201,6049,319092
